In [2]:
import pandas as pd
import numpy as np
from binance.client import Client
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None 

In [3]:
class Strategy():
    def __init__(self, client, symbols, timeframe, startdate = "2 year ago UTC", service_charge = 0.00075, money = 1000, storage = 0):
        self.client = client
        self.symbols = symbols
        self.timeframe = timeframe
        self.startdate = startdate
        self.service_charge = service_charge
        self.start_money = [money] * len(symbols)
        self.start_storage = [storage] * len(symbols)

        self.all_data = []
        for coin in symbols:
            bars = client.get_historical_klines(symbol=f'{coin}USDT',interval=timeframe,start_str=self.startdate)
            test_df = pd.DataFrame(bars[:],columns=["timestamp","open","high","low","close","volume", "close_time", "quote_asset_volume", "number_of_trade", "TBB", "TBQ", "ignore"])
            test_df["date"]=pd.to_datetime(test_df["timestamp"],unit="ms").astype(str)
            test_df = test_df.drop(["timestamp", "close_time", "quote_asset_volume", "number_of_trade", "TBB", "TBQ", "ignore"], axis=1)
            test_df["open"] = pd.to_numeric(test_df["open"])
            test_df["high"] = pd.to_numeric(test_df["high"])
            test_df["low"] = pd.to_numeric(test_df["low"])
            test_df["close"] = pd.to_numeric(test_df["close"])
            test_df["volume"] = pd.to_numeric(test_df["volume"])
            test_df["signal"] = ["None"] * len(test_df)
            self.all_data.append(test_df)

        self.profit = [0] * len(symbols)
    def print(self):
        print("Time frame", self.timeframe)
        print("Start from", self.startdate)

        for i in range(len(self.symbols)):
            print("=============")
            print(self.symbols[i], self.profit[i])

        print("=============")
        print("average", sum(self.profit) / len(self.profit))

    def decide_action(self):
        pass

    def back_test(self, type = "all in", fixed_money = None):
        for i in range(len(self.symbols)):
            money = self.start_money[i]
            storage = self.start_storage[i]
            for j in range(len(self.all_data[i])):
                if (self.all_data[i]["signal"][j] == "buy"):
                    if (type == "all in"):
                        if (storage < 0):
                            money -= -1 * storage * self.all_data[i]["close"][j]
                            storage = 0
                        storage += money / self.all_data[i]["close"][j] 
                        money = 0
                    else:
                        if (storage > 0):
                            money -= -1 * storage * self.all_data[i]["close"][j]
                            storage = 0
                        storage += fixed_money[i] / self.all_data[i]["close"][j] 
                        money -= fixed_money[i]
                elif (self.all_data[i]["signal"][j] == "sell"):
                    if (type == "all in"):
                        if (storage > 0):
                            money += storage * self.all_data[i]["close"][j]
                            storage = 0
                        storage -= money / self.all_data[i]["close"][j] 
                        money += money
                    else:
                        if (storage > 0):
                            money += storage * self.all_data[i]["close"][j]
                            storage = 0
                        storage -= fixed_money[i] / self.all_data[i]["close"][j] 
                        money += fixed_money[i]
            self.profit[i] = (money + storage * self.all_data[i]["close"][len(self.all_data[i]) - 1]) / self.start_money[i]

In [4]:
class EMA(Strategy):
    def decide_action(self, smooth = 2, short_period = 10, long_period = 100, stable_offset = 150):
        for i in range(len(self.all_data)):
            self.all_data[i]["short EMA"] = self.all_data[i]["close"].copy()
            self.all_data[i]["long EMA"] = self.all_data[i]["close"].copy()

            for j in range(1, len(self.all_data[i])):
                self.all_data[i]["short EMA"][j] = self.all_data[i]["short EMA"][j] * smooth / (short_period + 1) + self.all_data[i]["short EMA"][j - 1] * (1 - smooth / (short_period + 1))
                self.all_data[i]["long EMA"][j] = self.all_data[i]["long EMA"][j] * smooth / (long_period + 1) + self.all_data[i]["long EMA"][j - 1] * (1 - smooth / (long_period + 1))
            for j in range(stable_offset, len(self.all_data[i])):
                if (self.all_data[i]["short EMA"][j - 1] > self.all_data[i]["long EMA"][j - 1] and self.all_data[i]["short EMA"][j - 2] < self.all_data[i]["long EMA"][j - 2]):
                    self.all_data[i]["signal"][j] = "buy"
                elif (self.all_data[i]["short EMA"][j - 1] < self.all_data[i]["long EMA"][j - 1] and self.all_data[i]["short EMA"][j - 2] > self.all_data[i]["long EMA"][j - 2]):
                    self.all_data[i]["signal"][j] = "sell"


In [6]:
profits = []
count = 0
target_symbol = ["BTC", "ETH", "ADA", "BNB"]
time_frames = ["1d", "12h", "6h", "4h", "2h", "1h"]
time_frames = ["4h"]
start_str = "1 year ago UTC"
client = Client("", "")
res = []
for timeframe in time_frames:
    for short_period in range(13, 17):
        for long_period in range(95,111):
            a = EMA(client, target_symbol, timeframe, start_str)
            a.decide_action(short_period=short_period, long_period=long_period)
            a.back_test(type="fixed_money", fixed_money=[1000] * len(target_symbol))
            res.append([short_period, long_period, timeframe, sum(a.profit) / len(a.profit)])
            print("short", short_period, "long", long_period, "timeframe", timeframe, "average", sum(a.profit) / len(a.profit))

short 13 long 95 timeframe 4h average 1.9752962106391347
short 13 long 96 timeframe 4h average 1.9800874288163908
short 13 long 97 timeframe 4h average 2.002755472356385
short 13 long 98 timeframe 4h average 2.0049464548889655
short 13 long 99 timeframe 4h average 2.014301469922138
short 13 long 100 timeframe 4h average 1.9972087889375496
short 13 long 101 timeframe 4h average 1.9825851096260239
short 13 long 102 timeframe 4h average 1.9637811232876592
short 13 long 103 timeframe 4h average 1.978228037922379
short 13 long 104 timeframe 4h average 1.9929400070998058
short 13 long 105 timeframe 4h average 2.000721079863799
short 13 long 106 timeframe 4h average 1.9903573816793203
short 13 long 107 timeframe 4h average 1.9829385594318931
short 13 long 108 timeframe 4h average 1.9764145601351626
short 13 long 109 timeframe 4h average 1.965330930638617
short 13 long 110 timeframe 4h average 1.9085072256175053
short 14 long 95 timeframe 4h average 1.9977801824405361
short 14 long 96 timefram

In [ ]:
a.print()

Time frame 12h
Start from 2 year ago UTC
BTC 4.056921398629486
ETH 13.624799470601852
ADA 27.3147432338424
BNB 21.774236596878115
average 16.69267517498796
